In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
path = "/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset"

In [2]:
import os
import shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
!mkdir fish_dataset

In [4]:
fishes = []
for name in os.listdir(path):
    if os.path.isdir(os.path.join(path, name)):
        fishes.append(name)

In [5]:
for fish in fishes:
    fish_image_folder = os.path.join(path, fish, fish)
    shutil.copytree(fish_image_folder, os.path.join("./fish_dataset", fish))

In [6]:
print(os.listdir("./fish_dataset"))
print(os.listdir("./fish_dataset/Gilt-Head Bream")[:3])

['Hourse Mackerel', 'Striped Red Mullet', 'Red Sea Bream', 'Black Sea Sprat', 'Gilt-Head Bream', 'Sea Bass', 'Shrimp', 'Trout', 'Red Mullet']
['00826.png', '00133.png', '00731.png']


In [7]:
img_size = (224, 224)
batch_size = 32

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    "./fish_dataset/",
    labels="inferred",
    label_mode="categorical",  
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,  
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="training",
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    "./fish_dataset/",
    labels="inferred",
    label_mode="categorical",  
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,  
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="validation",
)


Found 9000 files belonging to 9 classes.
Using 7200 files for training.


2022-01-10 19:03:57.976992: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 19:03:58.087767: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 19:03:58.088516: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 19:03:58.091406: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 9000 files belonging to 9 classes.
Using 1800 files for validation.


In [ ]:
model = keras.Sequential([
    layers.Input((224, 224, 3)),
    
    layers.Conv2D(16, 3, activation="relu"),
    layers.BatchNormalization(), # regularization
    layers.MaxPooling2D(), 
    
    layers.Conv2D(32, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(64, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.2), # regularization
    layers.Dense(9, activation="softmax"),
])

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.0001),
    loss = keras.losses.CategoricalCrossentropy(),
    metrics = ["accuracy"],
)

In [ ]:
model.fit(ds_train, batch_size=64, epochs=5, verbose=2, validation_data=ds_validation)

In [16]:
mirrored_strategy = tf.distribute.MirroredStrategy()

2022-01-10 19:00:12.652307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 19:00:12.652831: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 19:00:12.653170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [8]:
from tensorflow.keras.applications import EfficientNetB0


inputs = layers.Input(shape=(224, 224, 3))
#x = img_augmentation(inputs)
outputs = EfficientNetB0(include_top=True, weights=None, classes=9)(inputs)

model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.summary()



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 9)                 4061100   
Total params: 4,061,100
Trainable params: 4,019,077
Non-trainable params: 42,023
_________________________________________________________________


In [9]:
epochs = 40  # @param {type: "slider", min:10, max:100}
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_validation, verbose=2)

Epoch 1/40


2022-01-10 19:05:36.117301: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-10 19:05:40.388726: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


225/225 - 85s - loss: 2.0889 - accuracy: 0.4176 - val_loss: 16.4672 - val_accuracy: 0.1072
Epoch 2/40
225/225 - 68s - loss: 0.7169 - accuracy: 0.7604 - val_loss: 8.8663 - val_accuracy: 0.2317
Epoch 3/40
225/225 - 68s - loss: 0.3431 - accuracy: 0.8871 - val_loss: 1.1215 - val_accuracy: 0.7439
Epoch 4/40
225/225 - 69s - loss: 0.1962 - accuracy: 0.9371 - val_loss: 0.2554 - val_accuracy: 0.9156
Epoch 5/40
225/225 - 69s - loss: 0.1193 - accuracy: 0.9608 - val_loss: 0.2692 - val_accuracy: 0.9317
Epoch 6/40
225/225 - 68s - loss: 0.1171 - accuracy: 0.9633 - val_loss: 0.3901 - val_accuracy: 0.8867
Epoch 7/40
225/225 - 68s - loss: 0.0781 - accuracy: 0.9731 - val_loss: 0.0671 - val_accuracy: 0.9739
Epoch 8/40
225/225 - 69s - loss: 0.0623 - accuracy: 0.9815 - val_loss: 0.0274 - val_accuracy: 0.9917
Epoch 9/40
225/225 - 67s - loss: 0.0781 - accuracy: 0.9758 - val_loss: 0.0914 - val_accuracy: 0.9672
Epoch 10/40
225/225 - 67s - loss: 0.0731 - accuracy: 0.9771 - val_loss: 0.6677 - val_accuracy: 0.8467

KeyboardInterrupt: 